<a href="https://colab.research.google.com/github/ManuelProy/digital-invoice/blob/main/digital-invoice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'factura:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4443305%2F7626757%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240214%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240214T235935Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1ef27e4103089bcc73ca04ce25767686cb839248e452ad8118908e70dd41368eaf771f6d7654ccd352bc9fa531f785705d4f38552f32804c19aa310874e2f731db80925acb754dd21417029c0bfa8c7775e837ee18b57bad76f8bb440896a83b634ee296cf798117fd99f65299f124c09d5c76bc133f62209868e6f7c100d8f0f626aaac3bd014f170b64c7d9dc272f69bad2043485f97bc92e63eb026bfedc198c768e3782fc273f2dbfdd0a7f8ea562d8685c24290d028770d464250988c1ed641793ae06572fee479125fc80fdc5e1abe77b0d991a0001d7abbad56adc9986eb2b50ff52e565bd90873c463a32d5f72065ae85685ce64d2888c890d0f4397,factura2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4443322%2F7626775%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240214%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240214T235936Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6c162b3ff9e45d3470e22a8fbd5b3e593783bb7021bac4014efbaa06521ca1126a173cdec132d20f478bd4349f179d5721670e39960c5045ec2152dbbb63a7c81a830d2099d8ece8e3e2184b6df04384b6d991abbe85530d8ee38e506d37047f6fc7eb6e4dbda928eff2e38d2e1234fba1471cf58532ef2791bb0cf479159d62f1613bc18e4575cccdf924cd4881f8521f9226ed6488d27a1d20ca167df71a57af1c4bedff04274749db76b4bfd65edf4eaff7a97eae47b36d926e4b3ab7e09ae086840beaa484d02cadd7c0b3b604946c29eafc096d260efebdbd489d205c00068f599cf2e1596dbc2b628e21d064fb588f2837a68d303c1f6d89ff7ecc6caa,factura3:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4443362%2F7626832%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240214%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240214T235936Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8f03c60aea447b725de26aac263b8db2bed3bc94928ee88781f525bfa1720958d40ff60af9ba5e258a3ba9506f7642d0aef4afcc4e56d7b034f3a74f20314b2a320b8a4c7399256332ca8d6207e0e280604eca6e219bb9759c54d8a86ca6af29f8ec35651815363629f9428b8d76c6f18d308294c65427f8025671abbb613c4680952de5e9b5ec2dcb54402eeae0a2b6f1f2a93c12d07aba889114a062c6731a10164dae08cf021d86c4954226bed8f9e4abf09d26e25585aefa2696e884c99fb5861d8eb806af7f04273f93fe577b1c0a091223d22f72c42fb03079f66dc3d0ae7dd8e0c7cef7a916278f5c48948e597509f4966b9efc42e0cdf3362914978b,facturas:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4443592%2F7627160%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240214%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240214T235936Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D065b7b31841421a13b61aa6531654ff3213170443d00e9ea18bd4bd54fa0fdda3030b219b894adeac4ead96d31fca7de151ccab2f79a02b4db58b0e36066eb4da42b7d4c2ac2ccf69d59a7a4cbeab1e502ecf72e31f2b6a7652b498c666680ec868eb1c5eee49dacdcff37ed3167bfd866e833e28da6659b50b1d3f665c7972c81f6bf048ece6408fb09d4cf88b6cd7d154a0f4448315da0d004cc86382d8a288a951bd579268f05a94a9fc685cc83d314cb2964fb1b45485ddbdbde1703f3f1c92560390c82fc202cb2b991b801c0412e71556a1ac659db66425105941b640d5125f8f73af3d1700c4abd06c11dc81c726adc24d4bcc23f545485930664e0e8'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Receipt OCR notebook series: Problem definition

> Given a arbitrary photo containing a receipt, extract grand total number

![](https://storage.googleapis.com/www.forwardit.lv/kaggle/receipt_home_kaggle.png)

The solution is divided into smaller tasks:

1. Get a scanned version of receipt by restoring perspective (done in the [previous notebook](https://www.kaggle.com/dmitryyemelyanov/receipt-ocr-part-1-image-segmentation-by-opencv))
2. **Apply OCR to find all texts within image**
3. **Find grand total as the largest number among recognized texts**

> One should remember that some tasks could be solved with traditional computer vision algorithms

This work aims to address a common pitfall among machine learning practitioners: trying to solve simple problems with the complex tools.

# About this notebook

This is a **part two notebook** in the *Receipt OCR with OpenCV* series. [Previously](https://www.kaggle.com/dmitryyemelyanov/receipt-ocr-part-1-image-segmentation-by-opencv) we have extracted a scanned version of the receipt out of the image. This notebook deals with the second step of the process: reading text information from it.
* Locating text boxes on the image
* Extracting all the texts from the image
* Obtaining grand total as the largest floating point number among texts

Let's get started!

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pytesseract
import re

from pytesseract import Output

Defining helper methods:

In [ ]:
def plot_gray(image):
    plt.figure(figsize=(16,10))
    return plt.imshow(image, cmap='Greys_r')

In [ ]:
def plot_rgb(image):
    plt.figure(figsize=(16,10))
    return plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

We will use a scanned version of the receipt, which is the output of the [previous notebook in the series](https://www.kaggle.com/dmitryyemelyanov/receipt-ocr-part-1-image-segmentation-by-opencv):

In [ ]:
file_name = "/kaggle/input/facturas/ordinaria.jpg"
image = cv2.imread(file_name, cv2.IMREAD_GRAYSCALE)
plot_gray(image)

# Step 1: Text box detection

In [ ]:
d = pytesseract.image_to_data(image, output_type=Output.DICT)
n_boxes = len(d['level'])
boxes = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2RGB)
for i in range(n_boxes):
    (x, y, w, h) = (d['left'][i], d['top'][i], d['width'][i], d['height'][i])
    boxes = cv2.rectangle(boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)

plot_rgb(boxes)

In [ ]:
boxes[0]

# Step 2: Text recognition

In [ ]:
extracted_text = pytesseract.image_to_string(image)
print(extracted_text)
type(extracted_text)


# Step 3: Extracting invoice lines

We will use regular expression to extract all floating point numbers out of the all detected texts.

In [ ]:
patron = r'\b\d+,\d+\b'
resultados = re.findall(patron, extracted_text)

print(resultados)

Grand total is the largest one:

In [ ]:
def lines(extracted_text):
    lineas_con_numeros = [linea for linea in extracted_text.splitlines() if re.search(r'^(?=.*[a-zA-Z])(?=.*\d.*,\d).*$', linea)]
    diccionario = {f"Línea {i + 1}": linea for i, linea in enumerate(lineas_con_numeros)}

    return(diccionario)

In [ ]:
lines(extracted_text)



In [ ]:
import re

texto = "Línea sin decimales ni letras\nLínea con el número 42\nOtra línea con texto\n123,45"

lineas_filtradas = [linea for linea in texto.splitlines() if re.match(r'^(?=.*[a-zA-Z])(?=.*\d)|(?=[^a-zA-Z\d])$', linea)]

print(lineas_filtradas)


In [ ]:
diccionario

Feel free to experiment with other regular expressions to find VAT number, company name, date or even read receipt positions!

# About the Author

This notebook is published under the **Data Science DJ** initiative with the goal of giving you distilled pieces of valuable information, short and concise, easy to comprehend.

I spend a few hours every day to write a single post about a single concept. You can find them by:

* [Joining my Telegram channel](https://t.me/datasciencedj)
* [Following my LinkedIn tag](https://www.linkedin.com/feed/hashtag/?keywords=datasciencedj)

If this work gives you joy, or maybe even inspiration, please consider contributing to my [Patreon account](https://www.patreon.com/datasciencedj).
Thank you!

# Resources

1. [OCR a document, form, or invoice with Tesseract, OpenCV, and Python](https://www.pyimagesearch.com/2020/09/07/ocr-a-document-form-or-invoice-with-tesseract-opencv-and-python/) by Adrian Rosebrock
2. [Text Detection and Extraction using OpenCV and OCR](https://www.geeksforgeeks.org/text-detection-and-extraction-using-opencv-and-ocr/) by Anandh Jagadeesan
3. [A comprehensive guide to OCR with Tesseract, OpenCV and Python](https://nanonets.com/blog/ocr-with-tesseract/) by Filip Zelic & Anuj Sable